
Observed trend 1: There are significantly more purchases by male players than female playsers. 
Observed trend 2: Most of the purchases were by players between age 15 and 34.
Observed trend 3: The two most popular items, Betrayal Whisper of Grieving Widows and Arcane Gem, were not among the five most profitalbe items. 

In [24]:
# Import Dependencies
import pandas as pd
import json

# File paths
file_path = "purchase_data.json"

# Read .json data to a pandas Dataframe
dataframe = pd.read_json(file_path)

# Check that the two have been added
dataframe.tail()

,Age,Gender,Item ID,Item Name,Price,SN
775,22,Male,98,"Deadline, Voice Of Subtlety",3.62,Eural50
776,14,Male,104,Gladiator's Glaive,1.36,Lirtossa78
777,20,Male,117,"Heartstriker, Legacy of the Light",4.15,Tillyrin30
778,20,Male,75,Brutality Ivory Warmace,1.72,Quelaton80
779,23,Female,107,"Splitter, Foe Of Subtlety",3.61,Alim85


In [25]:
# Player Count
# Find number of players
players = dataframe["SN"].unique()
total_number_of_players = len(players)
total_players_df = pd.DataFrame({"Total Players":[total_number_of_players]})
total_players_df

,Total Players
0,573


In [26]:
# Purchasing Analysis (Total)
# Find number of unique items
unique_items = dataframe["Item ID"].unique()
number_of_unique_items = len(unique_items)

# Find the average purchase price
average_purchase_price = dataframe["Price"].mean()

# Find total number of purchases
total_number_of_purchases = dataframe["Age"].count()

# Find total revenue
total_revenue = dataframe["Price"].sum()

# Purchasing Analysis Results
purchasing_analysis = pd.DataFrame({"Number of Unique Items":[number_of_unique_items],
                                    "Average Purchase Price":["$"+str(round(average_purchase_price,2))],
                                    "Total Number of Purchases":[total_number_of_purchases],
                                    "Total Revenue":["$"+str(round(total_revenue,2))]})
purchasing_analysis

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,$2.93,183,780,$2286.33


In [27]:
# Gender Demographics
# Remove duplicate players
gender_df = dataframe.drop_duplicates(subset=["SN"])
# Count the number of each gender
gender_dict = gender_df["Gender"].value_counts().to_dict()
# Find the percent of each gender and round the number displayed to 2 decimal places
gender_percent_dict = {k: '%.2f' %(v / total_number_of_players) for k, v in gender_dict.items()}
# Create a DataFrame from the percentages
gender_result_df = pd.DataFrame.from_dict(gender_percent_dict,orient='index')
# Add the Total Count to the DataFrame
gender_result_df["Total Count"]=gender_dict.values()
# Change the column name from 0 to Percentage of Players
gender_result_df = gender_result_df.rename(columns={0:"Percentage of Players"})
gender_result_df

,Percentage of Players,Total Count
Male,0.81,465
Female,0.17,100
Other / Non-Disclosed,0.01,8


In [28]:
# Purchase Analysis (Gender)
# Group the dataframe by gender
gender_group = dataframe.groupby(["Gender"])
# Create a DataFrame for the count, average, and total value of the prices
gender_purchase_analysis = pd.DataFrame({"Purchase Count":gender_group["Price"].count(),
                                        "Average Purchase Price":gender_group["Price"].mean(),
                                        "Total Purchase Value":gender_group["Price"].sum(),
                                        "Normalized Totals":gender_group["Price"].sum()}) 
# Loop through and divide the purchase total by the number of each gender
for x in gender_dict.keys():
    gender_purchase_analysis.loc[x,"Normalized Totals"] = gender_purchase_analysis.loc[x,"Normalized Totals"]/gender_dict[x]

# Format the monetary values and reorder the columns
gender_purchase_analysis["Average Purchase Price"] = gender_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
gender_purchase_analysis["Total Purchase Value"] = gender_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
gender_purchase_analysis["Normalized Totals"] = gender_purchase_analysis["Normalized Totals"].map("${:.2f}".format)
gender_purchase_analysis = gender_purchase_analysis[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]
gender_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,$1867.68,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [29]:
# Age Demographics
bins = [0,9,14,19,24,29,34,39,dataframe["Age"].max()]
bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
age_dataframe = dataframe
age_dataframe["Age Range"] = pd.cut(age_dataframe["Age"], bins, labels=bin_names)
age_groups = age_dataframe.groupby("Age Range")

# DataFrame with duplacates removed
normalized_age_dataframe = gender_df
normalized_age_dataframe["Age Range"] = pd.cut(normalized_age_dataframe["Age"], bins, labels=bin_names)
normalized_age_groups = normalized_age_dataframe.groupby("Age Range")

age_demographics_df = pd.DataFrame({"Purchase Count":age_groups["Price"].count(),
                                        "Average Purchase Price":age_groups["Price"].mean(),
                                        "Total Purchase Value":age_groups["Price"].sum(),
                                   "Normalized Totals":age_groups["Price"].sum()}) 

# Count the number of each age range
age_range_dict = normalized_age_groups["Age"].count().to_dict()
# Loop through and divide the purchase total by the number of each gender
for x in age_range_dict.keys():
    age_demographics_df.loc[x,"Normalized Totals"] = age_demographics_df.loc[x,"Normalized Totals"]/age_range_dict[x]

# Format the monetary values and reorder the columns
age_demographics_df["Average Purchase Price"] = age_demographics_df["Average Purchase Price"].map("${:.2f}".format)
age_demographics_df["Total Purchase Value"] = age_demographics_df["Total Purchase Value"].map("${:.2f}".format)
age_demographics_df["Normalized Totals"] = age_demographics_df["Normalized Totals"].map("${:.2f}".format)
age_demographics_df = age_demographics_df[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]
age_demographics_df

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


In [30]:
# Top Spenders
top_spenders = dataframe.groupby(["SN"])
top_spender_df = pd.DataFrame({"Purchase Count":top_spenders["Price"].count(),
                                        "Average Purchase Price":top_spenders["Price"].mean(),
                                        "Total Purchase Value":top_spenders["Price"].sum()})
top_spender_df = top_spender_df.sort_values(["Total Purchase Value"],ascending=False)

# Format the monetary values and reorder the columns
top_spender_df["Average Purchase Price"] = top_spender_df["Average Purchase Price"].map("${:.2f}".format)
top_spender_df["Total Purchase Value"] = top_spender_df["Total Purchase Value"].map("${:.2f}".format)
top_spender_df = top_spender_df[["Purchase Count","Average Purchase Price","Total Purchase Value"]]
top_spender_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [37]:
#Most Popular Items
popular_item = dataframe.groupby(["Item ID","Item Name"])
popular_item_df = pd.DataFrame({"Purchase Count":popular_item["Price"].count(),
                                        "Item Price":popular_item["Price"].mean(),
                                        "Total Purchase Value":popular_item["Price"].sum()})
most_popular_item_df = popular_item_df.sort_values(["Purchase Count"],ascending=False)

# Format the monetary values and reorder the columns
most_popular_item_df["Item Price"] = most_popular_item_df["Item Price"].map("${:.2f}".format)
most_popular_item_df["Total Purchase Value"] = most_popular_item_df["Total Purchase Value"].map("${:.2f}".format)
most_popular_item_df = most_popular_item_df[["Purchase Count","Item Price","Total Purchase Value"]]
most_popular_item_df.head(6)
# There is a four way tie between the second most popular item

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41
34,Retribution Axe,9,$4.14,$37.26


In [32]:
#Most Profitable Items
most_profitable_item_df = popular_item_df.sort_values(["Total Purchase Value"],ascending=False)

# Format the monetary values and reorder the columns
most_profitable_item_df["Item Price"] = most_profitable_item_df["Item Price"].map("${:.2f}".format)
most_profitable_item_df["Total Purchase Value"] = most_profitable_item_df["Total Purchase Value"].map("${:.2f}".format)
most_profitable_item_df = most_profitable_item_df[["Purchase Count","Item Price","Total Purchase Value"]]
most_profitable_item_df.head(5)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
